In [3]:
from pyspark.sql.types import *

activity_code_schema = StructType([
    StructField("activity_code",IntegerType()),
    StructField("activity",StringType())    
]) 

activity_df= spark.read.csv("s3a://indiantimedataset/DataSet/AcitivityCodes.csv",schema=activity_code_schema,header=True)
activity_df.printSchema()
activity_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- activity_code: integer (nullable = true)
 |-- activity: string (nullable = true)

+-------------+--------------------+
|activity_code|            activity|
+-------------+--------------------+
|          111| Ploughing, prepa...|
|          112| Sewing. planting...|
|          113| Application of m...|
|          114|            Weeding |
|          115| Supervision of w...|
+-------------+--------------------+
only showing top 5 rows

In [4]:
state_code_schema = StructType([
    StructField('state',StringType()),
    StructField('state_code',IntegerType())
])
state_df= spark.read.csv("s3a://indiantimedataset/DataSet/StateCodes.csv",header=True,schema=state_code_schema)
state_df.printSchema()
state_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- state: string (nullable = true)
 |-- state_code: integer (nullable = true)

+----------------+----------+
|           state|state_code|
+----------------+----------+
|        Haryana |         1|
| Madhya Pradesh |         2|
|        Gujarat |         3|
|          Orissa|         4|
|      Tamil Nadu|         5|
+----------------+----------+
only showing top 5 rows

In [5]:
from pyspark.sql import functions as F
activities_schema = StructType([
    StructField('_c0',StringType()),
     StructField('Key_membno',StringType()),
     StructField('Key_hhold',StringType()),
     StructField('Rec_id',StringType()),
     StructField('Schedule_ID',StringType()),
     StructField('Schedule_No',IntegerType()),
     StructField('sector',IntegerType()),
     StructField('subround',IntegerType()),
     StructField('subsample',IntegerType()),
     StructField('State',IntegerType()),
     StructField('District',IntegerType()),
     StructField('district_class',IntegerType()),
     StructField('Tehsil_Town',IntegerType()),
     StructField('stratum',IntegerType()),
     StructField('Vill_Blk',IntegerType()),
     StructField('sub_blk',IntegerType()),
     StructField('stage2_stratum',IntegerType()),
     StructField('Hholdno',IntegerType()),
     StructField('membno',IntegerType()),
    StructField('age',IntegerType()),
    StructField("type_of_day",IntegerType()),
    StructField("hour_serial_no",IntegerType()),
    StructField("activity_serial_no",IntegerType()),
    StructField("hour_from",IntegerType()),
    StructField("hour_to",IntegerType()),
     StructField("multiple_activity",IntegerType()),
     StructField("actual_time_spent_minutes",IntegerType()),
     StructField("within_outside_household",IntegerType()),
    StructField("activity",IntegerType()),   
    
])

activities_df = spark.read.csv("s3a://indiantimedataset/DataSet/time_use_partial.csv",header=True,schema=activities_schema)
activities_df = activities_df.select('Key_membno','Key_hhold','State','District','district_class','age','type_of_day','hour_from','hour_to',
                                  'multiple_activity','actual_time_spent_minutes','within_outside_household','activity')

activities_df.printSchema()
activities_df.show(5)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Key_membno: string (nullable = true)
 |-- Key_hhold: string (nullable = true)
 |-- State: integer (nullable = true)
 |-- District: integer (nullable = true)
 |-- district_class: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- type_of_day: integer (nullable = true)
 |-- hour_from: integer (nullable = true)
 |-- hour_to: integer (nullable = true)
 |-- multiple_activity: integer (nullable = true)
 |-- actual_time_spent_minutes: integer (nullable = true)
 |-- within_outside_household: integer (nullable = true)
 |-- activity: integer (nullable = true)

+----------+-----------+-----+--------+--------------+---+-----------+---------+-------+-----------------+-------------------------+------------------------+--------+
|Key_membno|  Key_hhold|State|District|district_class|age|type_of_day|hour_from|hour_to|multiple_activity|actual_time_spent_minutes|within_outside_household|activity|
+----------+-----------+-----+--------+--------------+---+-----------+---------+---

In [13]:
#data cleaning
activities_df = activities_df.filter((activities_df['age'] != 0) & (activities_df['state'] != 0) | (activities_df['state'] != None) 
                                     & (activities_df['activity'] != 0))

activities_df.write.partitionBy('state','age','activity').parquet("s3a://indiantimedataset/output/time_use_analysis_part.parquet")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o236.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:174)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:173)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:169)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:197)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOpe

In [27]:

#Data analysis

time_use_df= spark.read.parquet("s3a://indiantimedataset/output/time_use_analysis_part.parquet");

time_use_df.createOrReplaceTempView("timeuse")
state_df.createOrReplaceTempView("state")
activity_df.createOrReplaceTempView("activitycode")

#get actual state name and activity name along with code
time_use_data = spark.sql("""select *,s.State state_name,a.Activity as activity_name from timeuse t inner join activitycode a on t.activity = a.activity_code \
                             inner join state s on s.state_code = t.State""")
time_use_data.show(5)
time_use_data.createOrReplaceTempView("time_use_data")



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----------+--------+--------------+-----------+---------+-------+-----------------+-------------------------+------------------------+-----+---+--------+-------------+--------------------+----------+----------+----------+--------------------+
|Key_membno|  Key_hhold|District|district_class|type_of_day|hour_from|hour_to|multiple_activity|actual_time_spent_minutes|within_outside_household|state|age|activity|activity_code|            activity|     state|state_code|state_name|       activity_name|
+----------+-----------+--------+--------------+-----------+---------+-------+-----------------+-------------------------+------------------------+-----+---+--------+-------------+--------------------+----------+----------+----------+--------------------+
| 1.422E+12|14220001102|       2|             1|          1|        4|      5|                2|                       60|                       1|    6| 40|     911|          911|  Night sleep/ess...| Meghalaya|         6| Meghalay

In [29]:
#get unique activities on each state
unique_activity_per_state = spark.sql("""select state_name,activity_name from time_use_data group by state_name,activity_name""")
unique_activity_per_state.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------------+
|state_name|       activity_name|
+----------+--------------------+
| Meghalaya| Ploughing, prepa...|
| Meghalaya|  Assisting depen...|
| Meghalaya|  Attendance at p...|
| Meghalaya| Collection of ra...|
| Meghalaya|  Night sleep/ess...|
+----------+--------------------+
only showing top 5 rows

In [26]:
#get the popular activity
popular_activity =spark.sql("""select activity_name,count(activity_name) activity_count from time_use_data group by activity_name order by activity_count desc limit 1""")
popular_activity.show(1)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------+
|       activity_name|activity_count|
+--------------------+--------------+
|  Night sleep/ess...|           425|
+--------------------+--------------+